In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time
import requests
import gzip
import os

fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))
train_images = train_images / 255.0
test_images = test_images / 225.0

In [6]:
def get_testset():
    url = 'https://courses.engr.illinois.edu/ece498icc/sp2019/lab2_request_dataset.php'
    values = {'request': 'testdata', 'netid':'smeng10', 'team':'msp'}
    r = requests.post(url, data=values, allow_redirects=True)
    filename = r.url.split("/")[-1]
    testset_id = filename.split(".")[0].split("_")[-1]
    with open(filename, 'wb') as f: 
        f.write(r.content)
    return load_dataset(filename), testset_id

def load_dataset(path):
    num_img = 1000
    with gzip.open(path, 'rb') as infile:
        data = np.frombuffer(infile.read(), dtype=np.uint8).reshape(num_img, 784)
    return data

def send_result(pred, testset_id,latency):
    url = 'https://courses.engr.illinois.edu/ece498icc/sp2019/lab2_request_dataset.php'
    values = {'request': 'verify', 
              'netid':'smeng10', 
              'team':'msp', 
              'testset_id' : testset_id, 
              'prediction' : pred, 
              'latency': latency}
    r = requests.post(url, data=values, allow_redirects=True)
    return r

def create_keras_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=3, kernel_size=5, padding='valid', activation='relu', input_shape=(28,28,1))) 
    model.add(tf.keras.layers.MaxPooling2D(strides=2))
    model.add(tf.keras.layers.Conv2D(filters=3, kernel_size=3, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(strides=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(50, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    model.summary()
    return model

def keras_test(ckpt, test_img):
    #KERAS checkpoint
    if ckpt == "FashionNet/":
        model = create_keras_model()
    elif ckpt == "FashionNetModified/":
        model = create_keras_model_modified()
    elif ckpt == "customized/":
        model = None
    model.load_weights(ckpt)
    
    start = time.time()
    pred = model.predict(test_img)
    end = time.time()
    
    pred = ''.join(list(map(lambda x: str(x.argmax()),pred)))
    return pred,calc_latency(start,end)

def autotest(ckpt):
    data = get_testset()
    imgs = data[0].reshape((data[0].shape[0], 28, 28, 1))/255.0
    #test keras
    pred, latency = keras_test(ckpt,imgs)
    r = send_result(pred=pred, testset_id=data[1], latency=latency)
    acc = int(r.text)/1000
    Score = acc / (5 * latency)
    print(f'checkpoint {ckpt} score is {Score}, its accuracy is {acc}, latency is {latency}')
    
def calc_latency(start,end):
    return end-start

# 1.1 Latency evaluation of your FashionNet in lab2 running on Raspberry Pi. 

In [7]:
r = autotest("FashionNet/")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 24, 24, 3)         78        
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 3)         84        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 3)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 108)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10900     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                5050      
__________

# 1.2 Implement a modified FashionNet

In [8]:
def create_keras_model_modified():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=3, kernel_size=5, padding='valid', activation='relu', input_shape=(28,28,1))) 
    model.add(tf.keras.layers.MaxPooling2D(strides=2))
    model.add(tf.keras.layers.DepthwiseConv2D(kernel_size=3, padding='same', activation='relu'))
    model.add(tf.keras.layers.SeparableConv2D(filters=3, kernel_size=1, padding='valid', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(strides=2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(50, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    model.summary()
    return model

In [9]:
new_model = create_keras_model_modified()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 24, 24, 3)         78        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 3)         0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 12, 12, 3)         30        
_________________________________________________________________
separable_conv2d (SeparableC (None, 12, 12, 3)         15        
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 3)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 108)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10900     
__________

In [11]:
history = new_model.fit(train_images, train_labels, validation_split=0.1, batch_size=64, epochs=10)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 14s 253us/step - loss: 0.7985 - acc: 0.7041 - val_loss: 0.6096 - val_acc: 0.7570
Epoch 2/10
54000/54000 [==============================] - 13s 241us/step - loss: 0.5311 - acc: 0.8048 - val_loss: 0.4977 - val_acc: 0.8192
Epoch 3/10
54000/54000 [==============================] - 13s 243us/step - loss: 0.4780 - acc: 0.8250 - val_loss: 0.4761 - val_acc: 0.8257
Epoch 4/10
54000/54000 [==============================] - 13s 237us/step - loss: 0.4438 - acc: 0.8387 - val_loss: 0.4881 - val_acc: 0.8202
Epoch 5/10
54000/54000 [==============================] - 13s 239us/step - loss: 0.4201 - acc: 0.8482 - val_loss: 0.4181 - val_acc: 0.8457
Epoch 6/10
54000/54000 [==============================] - 13s 238us/step - loss: 0.4013 - acc: 0.8530 - val_loss: 0.4109 - val_acc: 0.8440
Epoch 7/10
54000/54000 [==============================] - 13s 243us/step - loss: 0.3869 - acc: 0.8580 - val_loss: 0.3

In [19]:
new_model.save_weights(filepath='FashionNetModified/')


Consider using a TensorFlow optimizer from `tf.train`.


In [21]:
autotest("FashionNetModified/")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 24, 24, 3)         78        
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 12, 3)         0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 12, 12, 3)         30        
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 12, 12, 3)         15        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 6, 3)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 108)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 100)               10900     
__________

In [ ]:
autotest()